# Libraries

If you want to know about the packages, they usually have good documentation on what they can do and how to do it. Here's sklearn:
https://scikit-learn.org/stable/

In [ ]:
import pandas as pd #working with datasets

import xgboost as xgb #xgboost algo

#sklearn has many ML packages
import sklearn as sk
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import SGDRegressor as GD
from sklearn.metrics import mean_squared_error as MSE, r2_score as r2
from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV

# Functions

Here are some valuable functions that I've used.

In [1]:
#pass the regressor, and regressand and it will produce a scatter plot with LR fit line. Good for visualizing correlation and fit.
def LRgraph(regressor, regressand):

    model = LR(normalize=True)
    model.fit(regressor,regressand)
    plt.scatter(regressor,regressand,color='g')
    plt.plot(regressor,model.predict(regressor),color='k')
    plt.show()
    
#builds a linear regression model
def BuildModel(df, regressand, regressors_list, Metric):
    
    regressors = df.columns.intersection(regressors_list)

    rowCount = df.shape[0]
    trainNum = round(rowCount * 0.8)

    train = df[0:trainNum]
    test = df[trainNum:]

    X_train = train[regressors]
    y_train = train[regressand]
    X_test = test[regressors]
    y_test = test[regressand]
    
    first_term = np.linalg.inv(np.dot(np.transpose(X_train),X_train))
    second_term = np.dot(np.transpose(X_train),y_train)

    ols_estimation = np.dot(first_term, second_term)

    lr = LR()
    lr.fit(X_train,y_train)
    lr_coeff = pd.DataFrame(lr.coef_, X_train.columns, columns=['Coefficient'])

    y_pred = lr.predict(X_test)

    if Metric == 'MSE':
        print(MSE(y_test, y_pred))
    elif Metric == 'r2':
        print(r2_score(y_test, y_pred))
    else:
        both = 'r2: ' + str(r2_score(y_test, y_pred)) + ', MSE: ' + str(MSE(y_test, y_pred))
        print(both)
        
    return lr

#builds a linear regression model, and returns the coefficients for each variable
def FindModelCoefficients(df, regressand, regressors_list):
    
    regressors = df.columns.intersection(regressors_list)

    rowCount = df.shape[0]
    trainNum = round(rowCount * 0.8)

    train = df[0:trainNum]
    test = df[trainNum:]

    X_train = train[regressors]
    y_train = train[regressand]
    X_test = test[regressors]
    y_test = test[regressand]
    
    first_term = np.linalg.inv(np.dot(np.transpose(X_train),X_train))
    second_term = np.dot(np.transpose(X_train),y_train)

    ols_estimation = np.dot(first_term, second_term)

    lr = LR()
    lr.fit(X_train,y_train)
    lr_coeff = pd.DataFrame(lr.coef_, X_train.columns, columns=['Coefficient'])

    y_pred = lr.predict(X_test)

    return lr_coeff

#builds a XGBoost model
def BuildXGBoost(df, regressand_list, regressors_list, searchType='Randomized', method='Tree'):
    
    regressors = df.columns.intersection(regressors_list)
    regressand = df.columns.intersection(regressand_list)
    X = df[regressors]
    y = df[regressand]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)
    
    params = {
    'n_estimators': [50,100],
    'max_depth': [None, 3, 5, 7, 9],
    'eta': [0.5, 1, 2, 3]
    }
    
    if method == 'Tree':
        objectiveValue = 'reg:squarederror'
        boosterValue = 'gbtree'
    else:
        objectiveValue = 'reg:linear'
        boosterValue = 'gblinear'
    
    #original
    if searchType == 'Grid':
        search = GridSearchCV(xgb.XGBRegressor(objective=objectiveValue,booster=boosterValue,learning_rate=1),params,n_jobs=-1)
    else:
        search = RandomizedSearchCV(xgb.XGBRegressor(objective=objectiveValue,booster=boosterValue,learning_rate=1),params,n_jobs=-1)

    search.fit(X_train, y_train)
    
    print("Test  R2 Score : %.2f"%search.score(X_test, y_test))
    print("Train R2 Score : %.2f"%search.score(X_train, y_train))

    print("Best Params : ", search.best_params_)
    
    return search


In [ ]:
#pull results from SQL
server = 'HFWDBRAT04PH'
db = 'RMCCDev'
query = """
        SELECT * FROM RMCCDev.EventsOptimization.Results
        where DateType = 0 and attendance > 0
        """
resultsPull = PullFromSQL(server, db, query)

# ML Modeling Example

In [ ]:
#assigning my dataset. you'll have to assign whatever data you are using
attendance_df = attendancePull.copy()

regressand_list = ['attendance']
regressors_list = ['Capacity','Rivalry','Stadium_Avg_OccPerc','GoodnessMetric','TravelMetric','neutral_site']
Metric = 'Randomized' #Grid/Randomized
Method = 'Tree' #Tree/Linear

#This builds a model using the "def BuildXGBoost" function above.
AttendanceModel = BuildXGBoost(attendance_df, regressand_list, regressors_list, Metric, Method)

In [ ]:
regressand = 'attendance'
regressors_list = ['Capacity','Rivalry','Stadium_Avg_OccPerc','GoodnessMetric','TravelMetric','neutral_site']
Metric = 'r2'

#This builds a LR model using the "def BuildModel" function above.
AttendanceModelLR = BuildModel(attendance_df, regressand, regressors_list, Metric)

Now, you can use those models to produce predictions

In [ ]:
#fs_df is the future schedule dataset. X_proj1 is the future schedule with only the regressor columns.
X_proj1 = fs_df[['neutral_site','Capacity','Rivalry','Stadium_Avg_OccPerc','GoodnessMetric','TravelMetric']].copy()

#make XGBoost predictions
y_proj1 = AttendanceModel.predict(X_proj1)
#make LR predictions
y_proj2 = AttendanceModelLR.predict(X_proj1)

#assign predictions as columns in the future schedule dataset
fs_df['preds1'] = y_proj1
fs_df['preds2'] = y_proj2

In [ ]:
#take a look!
fs_df